In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('arrests_usa_juvenile.csv')
df.head()

,id,year,state_abbr,offense_code,offense_name,agencies,population,total_male,total_female,m_0_9,...,f_13_14,f_15,f_16,f_17,race_agencies,race_population,white,black,asian_pacific_islander,american_indian
0,1081,2016,NaN,ASR_ARSON,Arson,13310,264534532,1760,328,117.0,...,114.0,66.0,39.0,43,12581,263887632,1436,516,41.0,48.0
1,1082,2016,NaN,ASR_AST,Aggravated Assault,13310,264534532,16997,5918,141.0,...,1565.0,1171.0,1337.0,1361,12581,263887632,12370,9736,296.0,364.0
2,1083,2016,NaN,ASR_AST_SMP,Simple Assault,13310,264534532,66360,38712,1043.0,...,11210.0,8036.0,8264.0,7590,12581,263887632,59778,41923,1130.0,1378.0
3,1084,2016,NaN,ASR_BRG,Burglary,13310,264534532,23307,3071,167.0,...,727.0,618.0,689.0,833,12581,263887632,14413,11082,368.0,362.0
4,1085,2016,NaN,ASR_CUR,Curfew and Loitering Law Violations,13310,264534532,19218,8319,110.0,...,1991.0,1887.0,2267.0,1701,12581,263887632,15468,11045,382.0,429.0


In [3]:
df.columns

Index(['id', 'year', 'state_abbr', 'offense_code', 'offense_name', 'agencies',
       'population', 'total_male', 'total_female', 'm_0_9', 'm_10_12',
       'm_13_14', 'm_15', 'm_16', 'm_17', 'f_0_9', 'f_10_12', 'f_13_14',
       'f_15', 'f_16', 'f_17', 'race_agencies', 'race_population', 'white',
       'black', 'asian_pacific_islander', 'american_indian'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      690 non-null    int64  
 1   year                    690 non-null    int64  
 2   state_abbr              0 non-null      float64
 3   offense_code            690 non-null    object 
 4   offense_name            690 non-null    object 
 5   agencies                690 non-null    int64  
 6   population              690 non-null    int64  
 7   total_male              690 non-null    int64  
 8   total_female            690 non-null    int64  
 9   m_0_9                   654 non-null    float64
 10  m_10_12                 685 non-null    float64
 11  m_13_14                 690 non-null    int64  
 12  m_15                    690 non-null    int64  
 13  m_16                    690 non-null    int64  
 14  m_17                    690 non-null    in

In [5]:
# 'dropeamos' la columna state_abbr que esta llena de nulos

df = df.drop('state_abbr', axis=1)
df

,id,year,offense_code,offense_name,agencies,population,total_male,total_female,m_0_9,m_10_12,...,f_13_14,f_15,f_16,f_17,race_agencies,race_population,white,black,asian_pacific_islander,american_indian
0,1081,2016,ASR_ARSON,Arson,13310,264534532,1760,328,117.0,368.0,...,114.0,66.0,39.0,43,12581,263887632,1436,516,41.0,48.0
1,1082,2016,ASR_AST,Aggravated Assault,13310,264534532,16997,5918,141.0,1406.0,...,1565.0,1171.0,1337.0,1361,12581,263887632,12370,9736,296.0,364.0
2,1083,2016,ASR_AST_SMP,Simple Assault,13310,264534532,66360,38712,1043.0,7406.0,...,11210.0,8036.0,8264.0,7590,12581,263887632,59778,41923,1130.0,1378.0
3,1084,2016,ASR_BRG,Burglary,13310,264534532,23307,3071,167.0,1474.0,...,727.0,618.0,689.0,833,12581,263887632,14413,11082,368.0,362.0
4,1085,2016,ASR_CUR,Curfew and Loitering Law Violations,13310,264534532,19218,8319,110.0,1184.0,...,1991.0,1887.0,2267.0,1701,12581,263887632,15468,11045,382.0,429.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,446,1994,ASR_STP,Stolen Property,10713,208091172,32205,4105,211.0,1677.0,...,1150.0,905.0,937.0,863,10282,207684879,21242,14077,587.0,364.0
686,447,1994,ASR_SUS,Suspicion,10713,208091172,1389,330,30.0,104.0,...,99.0,76.0,64.0,58,10282,207684879,1171,521,5.0,22.0
687,448,1994,ASR_VAG,Vagrancy,10713,208091172,2979,683,15.0,127.0,...,206.0,143.0,159.0,149,10282,207684879,2599,1010,40.0,13.0
688,449,1994,ASR_VAN,Vandalism,10713,208091172,110175,12616,5621.0,16102.0,...,4161.0,2272.0,2050.0,1864,10282,207684879,98391,20722,2073.0,1257.0


In [6]:
columns = df.columns

In [7]:
male_to_replace = [name for name in columns
                  if name.startswith('m')]
male_to_replace

['m_0_9', 'm_10_12', 'm_13_14', 'm_15', 'm_16', 'm_17']

In [8]:
replacement = [re.sub('^m', 'male', name) 
              for name in male_to_replace]
replacement

['male_0_9', 'male_10_12', 'male_13_14', 'male_15', 'male_16', 'male_17']

In [9]:
male_dict_names = dict(zip(male_to_replace, replacement))
male_dict_names

{'m_0_9': 'male_0_9',
 'm_10_12': 'male_10_12',
 'm_13_14': 'male_13_14',
 'm_15': 'male_15',
 'm_16': 'male_16',
 'm_17': 'male_17'}

In [10]:
df.rename(columns=male_dict_names, inplace=True)

In [11]:
df.columns

Index(['id', 'year', 'offense_code', 'offense_name', 'agencies', 'population',
       'total_male', 'total_female', 'male_0_9', 'male_10_12', 'male_13_14',
       'male_15', 'male_16', 'male_17', 'f_0_9', 'f_10_12', 'f_13_14', 'f_15',
       'f_16', 'f_17', 'race_agencies', 'race_population', 'white', 'black',
       'asian_pacific_islander', 'american_indian'],
      dtype='object')

In [12]:
columns = df.columns

In [13]:
female = '^f'
female_to_replace = [name for name in columns
                    if re.findall(female, name)]
female_to_replace

['f_0_9', 'f_10_12', 'f_13_14', 'f_15', 'f_16', 'f_17']

In [14]:
replacement = [re.sub('^f', 'female', name) 
              for name in female_to_replace]
replacement

['female_0_9',
 'female_10_12',
 'female_13_14',
 'female_15',
 'female_16',
 'female_17']

In [15]:
female_dict_names = dict(zip(female_to_replace, replacement))
female_dict_names

{'f_0_9': 'female_0_9',
 'f_10_12': 'female_10_12',
 'f_13_14': 'female_13_14',
 'f_15': 'female_15',
 'f_16': 'female_16',
 'f_17': 'female_17'}

In [16]:
df.rename(columns=female_dict_names, inplace=True)

In [17]:
df.columns

Index(['id', 'year', 'offense_code', 'offense_name', 'agencies', 'population',
       'total_male', 'total_female', 'male_0_9', 'male_10_12', 'male_13_14',
       'male_15', 'male_16', 'male_17', 'female_0_9', 'female_10_12',
       'female_13_14', 'female_15', 'female_16', 'female_17', 'race_agencies',
       'race_population', 'white', 'black', 'asian_pacific_islander',
       'american_indian'],
      dtype='object')

In [18]:
# existen columnas con valores nulos.
# estos valores nulos significa que hubo 0 arrestos para esa 'categoria'.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      690 non-null    int64  
 1   year                    690 non-null    int64  
 2   offense_code            690 non-null    object 
 3   offense_name            690 non-null    object 
 4   agencies                690 non-null    int64  
 5   population              690 non-null    int64  
 6   total_male              690 non-null    int64  
 7   total_female            690 non-null    int64  
 8   male_0_9                654 non-null    float64
 9   male_10_12              685 non-null    float64
 10  male_13_14              690 non-null    int64  
 11  male_15                 690 non-null    int64  
 12  male_16                 690 non-null    int64  
 13  male_17                 690 non-null    int64  
 14  female_0_9              594 non-null    fl

In [19]:
# reemplazar nulos por 0 en todo el df
df = df.fillna(0)

In [20]:
# ahora tenemos valores 'float' donde deberia de ser 'int'
# ya que son personas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      690 non-null    int64  
 1   year                    690 non-null    int64  
 2   offense_code            690 non-null    object 
 3   offense_name            690 non-null    object 
 4   agencies                690 non-null    int64  
 5   population              690 non-null    int64  
 6   total_male              690 non-null    int64  
 7   total_female            690 non-null    int64  
 8   male_0_9                690 non-null    float64
 9   male_10_12              690 non-null    float64
 10  male_13_14              690 non-null    int64  
 11  male_15                 690 non-null    int64  
 12  male_16                 690 non-null    int64  
 13  male_17                 690 non-null    int64  
 14  female_0_9              690 non-null    fl

In [21]:
float_columns = []

for colname, coltype in df.dtypes.iteritems():
    if coltype == 'float64':
        float_columns.append(colname)
        
float_columns

['male_0_9',
 'male_10_12',
 'female_0_9',
 'female_10_12',
 'female_13_14',
 'female_15',
 'female_16',
 'asian_pacific_islander',
 'american_indian']

In [22]:
# convertir columnas 'float' a 'int'
df[float_columns] = df[float_columns].apply(np.int64)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      690 non-null    int64 
 1   year                    690 non-null    int64 
 2   offense_code            690 non-null    object
 3   offense_name            690 non-null    object
 4   agencies                690 non-null    int64 
 5   population              690 non-null    int64 
 6   total_male              690 non-null    int64 
 7   total_female            690 non-null    int64 
 8   male_0_9                690 non-null    int64 
 9   male_10_12              690 non-null    int64 
 10  male_13_14              690 non-null    int64 
 11  male_15                 690 non-null    int64 
 12  male_16                 690 non-null    int64 
 13  male_17                 690 non-null    int64 
 14  female_0_9              690 non-null    int64 
 15  female

In [24]:
df.to_csv('clean_arrests_usa_juvenile.csv')